### This notebook is for problem set 3 of the stochastic simulation course. We have picked the traveling salesman problem (TSP).

All graphs analyzed in this notebook are assumed to:
1. Be complete graphs (all nodes are adjacent to all other nodes)
2. Follow the triangle inequality (the shortest path between nodes is through the edge that connects them)
3. Symmetric (the path from A to B has the same length as B to A)

In [ ]:
import numpy as np
import pandas as pd

### Optimizing path length using simulated annealing and the Boltzmann distribution

In [96]:
def load_data(file_path: str) -> pd.DataFrame:
    """Load the the nodes as defined in the TSP-Configurations folders in a dataframe.

    Args:
        file_path: The path to the file to load the data from.

    Returns:
        Dataframe with columns "Node", "X" and "Y", representing the node and its coordinates.
    """
    with open(file_path, "r") as file:
        lines = file.readlines()

    # Find the data by detecting the text 'NODE_COORD_SECTION'
    start_idx = lines.index("NODE_COORD_SECTION\n") + 1

    # Extract data until EOF
    data = []
    for line in lines[start_idx:]:
        if line.strip() == "EOF":
            break
        data.append(line.strip())

    # Convert the extracted data into a DataFrame
    df = pd.DataFrame(data=[map(int, line.split()) for line in data], columns=["Node", "X", "Y"])

    return df


def load_solutions(file_path: str) -> np.ndarray:
    """Load the the optimal TSP solutions as defined in the TSP-Configurations folders in a dataframe.

    Args:
        file_path: The path to the file to load the solutions from.

    Returns:
        Array containing the optimal path of the TSP.
    """
    with open(file_path, "r") as file:
        lines = file.readlines()

    # Find the data by detecting the text 'TOUR_SECTION'
    start_idx = lines.index("TOUR_SECTION\n") + 1

    # Extract data until -1
    data = []
    for line in lines[start_idx:]:
        if line.strip() == "-1":
            break
        data.append(line.strip())

    # Convert the extracted data into a DataFrame
    df = np.array(data+[data[0]], dtype=int)

    return df


small_tsp_data_path = "TSP-Configurations/eil51.tsp.txt"
small_tsp_solution_path = "TSP-Configurations/eil51.opt.tour.txt"
medium_tsp_data_path = "TSP-Configurations/a280.tsp.tx"
medium_tsp_solution_path = "TSP-Configurations/a280.opt.tour.txt"
large_tsp_data_path = "TSP-Configurations/pcb442.tsp.tx"
medium_tsp_solution_path = "TSP-Configurations/pcb442.opt.tour.txt"

cities = load_data(small_tsp_data_path)
print(cities.head())
opt_path = load_solutions(small_tsp_solution_path)
print(opt_path)

   Node   X   Y
0     1  37  52
1     2  49  49
2     3  52  64
3     4  20  26
4     5  40  30
[ 1 22  8 26 31 28  3 36 35 20  2 29 21 16 50 34 30  9 49 10 39 33 45 15
 44 42 40 19 41 13 25 14 24 43  7 23 48  6 27 51 46 12 47 18  4 17 37  5
 38 11 32  1]


In [106]:
def generate_initial_path(graph_data: pd.DataFrame) -> np.ndarray:
    """Generate a solution of the TSP problem by taking the shortest path between each node.

    Args:
        graph_data: The dataset to generate the solution for.

    Returns:
        Array containing a (suboptimal) solution to the TSP.
    """
    # Extract nodes and coordinates
    nodes = graph_data["Node"].values
    coordinates = graph_data[["X", "Y"]].values

    # Initialize variables
    visited = []  # List to store the visitation order
    current_node = nodes[0]  # Start at the first node
    visited.append(current_node)

    # Create a mask for visited nodes
    visited_mask = np.zeros(len(nodes), dtype=bool)
    visited_mask[current_node - 1] = True  # Mark the starting node as visited

    while len(visited) < len(nodes):
        current_coords = coordinates[current_node - 1]
        # Calculate distances to all unvisited nodes
        distances = np.linalg.norm(coordinates - current_coords, axis=1)
        distances[visited_mask] = np.inf  # Exclude already visited nodes
        # Find the nearest unvisited node
        next_node = np.argmin(distances) + 1  # Convert back to 1-based index
        visited.append(next_node)
        visited_mask[next_node - 1] = True  # Mark the node as visited
        current_node = next_node

    # Complete the cycle by returning to the starting node
    visited.append(visited[0])

    return np.array(visited)


def tsp_cost(graph_data: pd.DataFrame, solution: np.ndarray) -> float:
    """Calculates the cost of a solution to the TSP, which is defined as the total length of the path.

        Args:
            graph_data: The dataset to generate the solution for.
            solution: The path of the solution to the TSP.

        Returns:
            The cost of the solution.
    """
    # Extract the node coordinates
    coordinates = graph_data[["X", "Y"]].values

    # Get the coordinates of nodes in the solution
    path_coords = coordinates[solution - 1]  # Adjust for 1-based indexing in solution

    # Calculate distances between consecutive nodes in the solution
    distances = np.linalg.norm(path_coords[:-1] - path_coords[1:], axis=1)

    # Return the total cost as the sum of distances
    return distances.sum()

initial_sol = generate_initial_path(cities)
print(initial_sol)
print(initial_sol.shape, opt_path.shape)
cost_initial_sol = tsp_cost(cities, initial_sol)
print(cost_initial_sol)
cost_optimal_sol = tsp_cost(cities, opt_path)
print(cost_optimal_sol)

    

[ 1 32 11 38  5 49  9 50 16  2 29 21 34 30 10 39 33 45 15 44 37 17  4 18
 47 12 46 51 27 48  6 14 25 13 41 19 42 40 24 23  7 26  8 31 28  3 20 35
 36 22 43  1]
(52,) (52,)
513.610006884723
429.983311983384
